# GRO620 - Activité procédurale 3

In [ ]:
# Préambule

import random as rng
import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive

## Filtrage linéaire - suite

### Q1.1

Soit cette image :

In [ ]:
img_q11_org  = cv2.imread("images_doc/proc2-q1-object.jpeg")
img_q11_mono = np.float32(cv2.cvtColor(img_q11_org, cv2.COLOR_BGR2GRAY)) / 255.
plt.imshow(img_q11_mono, cmap="gray")

Utilisez un filtre linéaire de Sobel (voir sections 3.2.1 et 4.2.1) pour faire ressortir les contours de cette image.

In [ ]:
sobel_h = np.array([
    [1, 2, 1],
    [0, 0, 0],
    [-1, -2, -1]
]) * (1/8.)
sobel_v = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
]) * (1/8.)

dst_h = cv2.filter2D(img_q11_mono, -1, sobel_h)
dst_v = cv2.filter2D(img_q11_mono, -1, sobel_v)

dst = np.sqrt(np.square(dst_h) + np.square(dst_v))

plt.imshow(dst, cmap="gray")

## Filtrage non-linéaire

### Q2.1

Soit cette image :

In [ ]:
img_q21_org = cv2.imread("images_doc/proc2-q3-sand.png")
img_q21_mono = cv2.cvtColor(img_q21_org, cv2.COLOR_BGR2GRAY)
plt.imshow(img_q21_mono, cmap="gray")

Utilisez un filtre **non-linéaire** pour tenter de retirer les points noirs dans l'image.

In [ ]:
img = cv2.cvtColor(img_q21_org, cv2.COLOR_BGR2RGB)

# First method
kernel = np.ones((5,5), np.uint8)
dilation = cv2.dilate(img, kernel, iterations = 1)

# Second method
median = cv2.medianBlur(img, 5)

simfig, plots = plt.subplots(1,3, figsize = (15,6))
plots[0].imshow(img)
plots[1].imshow(dilation)
plots[2].imshow(median)

## Chaîne de filtrage

### Q3.1

En reprenant la figure originale de la question Q1.1, appliquez une succession de filtres pour mieux isoler le contour de l'objet au centre de l'image.

In [ ]:
img_q31 = img_q11_mono
plt.imshow(img_q31, cmap="gray")

### Q3.2

Répétez l'exercice précédent, mais avec un filtre de Canny.

In [ ]:
img_q32 = img_q11_org

dim = 15
minVal = 150
maxVal = 200
img_q32_gaus = cv2.GaussianBlur(img_q32, (15,15), 0)
img_q32 = cv2.Canny(img_q32, 3*minVal, 3*maxVal)
img_q32_gaus = cv2.Canny(img_q32_gaus, minVal, maxVal)

simfig, plots = plt.subplots(1,2, figsize = (10,6))
plots[0].imshow(img_q32, cmap="gray")
plots[1].imshow(img_q32_gaus, cmap="gray")

## Analyse de formes

### Q4.1

Soit cette image :

In [ ]:
img_q41_org  = cv2.imread("images_doc/proc3-q4-tracks.jpeg")
img_q41_rgb = cv2.cvtColor(img_q41_org, cv2.COLOR_BGR2RGB)
plt.imshow(img_q41_rgb)

(Source de l'image originale : [Krivec Ales, Pexels](https://www.pexels.com/photo/clouds-cloudy-country-distance-552779/))

Trouvez et tracez les lignes dominantes à l'aide de la transformée de Hough.

In [ ]:
img_q41_mono = cv2.cvtColor(img_q41_rgb, cv2.COLOR_BGR2GRAY)

dim = 15
minVal = 0
maxVal = 40
lines = cv2.GaussianBlur(img_q41_mono, (15,15), 0)
lines = cv2.Canny(lines, minVal, maxVal)
lines = cv2.HoughLines(lines, 1, np.pi / 180, 150, None, 0, 0)

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv2.line(img_q41_mono, pt1, pt2, (0,0,255), 3, cv2.LINE_AA)

plt.imshow(img_q41_mono, cmap="gray")

### Q4.2

En reprenant l'image des questions Q1.1 et Q3.1, trouvez, tracez et afficher les coordonnées (en pixels) de la boîte (*bounding box*) contenant les contours de l'objet à l'aide de cv2.findContours()

In [ ]:
img_q41_out = img_q11_org.copy() # Pour le résultat
# img_q41_out = cv2.cvtColor(img_q41_out, cv2.COLOR_BGR2RGB)

threshold = 400

# Detect edges using Canny
canny_output = cv2.Canny(img_q41_out, threshold, threshold * 2)

# Find contours
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours
# drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    # cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)
    cv2.drawContours(img_q41_out, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)

idx = 0 
for cnt in contours:
    idx += 1
    x, y, w, h = cv2.boundingRect(cnt)
    roi = img_q41_out[y:y + h, x:x + w]
    cv2.rectangle(img_q41_out, (x, y) ,(x + w, y + h), (200, 0, 0), 2)
    # cv2.rectangle(drawing,(x,y),(x+w,y+h),(200,0,0),2)
    
    # Print position
    print(f"Bounding box #{idx}:")
    print(f"  X: pixel {x} (top corner)")
    print(f"  Y: pixel {y} (top corner)")
    print(f"  W: {w} pixels")
    print(f"  H: {h} pixels")
    
# Show in a window
# simfig, plots = plt.subplots(1,2)
# plots[0].imshow(img_q41_out)
# plots[1].imshow(drawing)
plt.imshow(img_q41_out)